In [1]:
import torch
import pathlib
import cv2
import PIL
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
import csv
from tqdm import tqdm
import utm
import rasterio
%matplotlib inline
import passion.util

from passion.segmentation import prediction

In [2]:
device = ('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
models_folder_path = pathlib.Path('../workflow/output/model')
rooftop_models_folder_path = models_folder_path / 'rooftop-segmentation'

model_inria_aug_10e = torch.load(str(rooftop_models_folder_path / 'inria-aug-10e.pth'), map_location=torch.device(device))
model_inria_aug_5e = torch.load(str(rooftop_models_folder_path / 'inria-raw-10e.pth'), map_location=torch.device(device))
model_inria_raw_10e = torch.load(str(rooftop_models_folder_path / 'inria-aug-5e.pth'), map_location=torch.device(device))
model_inria_raw_5e = torch.load(str(rooftop_models_folder_path / 'inria-raw-5e.pth'), map_location=torch.device(device))

In [4]:
inria_val_folder = pathlib.Path('/storage/internal/home/r-pueblas/projects/rooftop-segmentation-datasets/data/INRIA/output/val')
inria_val_folder_image = inria_val_folder / 'image'
inria_val_folder_label = inria_val_folder / 'label'

rid_test_folder = pathlib.Path('/storage/internal/home/r-pueblas/projects/rooftop-segmentation-datasets/data/RID/output/masks_segments_reviewed/test')
rid_test_folder_image = rid_test_folder / 'image'
rid_test_folder_label = rid_test_folder / 'label'

nrw_path = pathlib.Path('/storage/internal/data/s-risch/exchange/shruthi/')
nrw_csv = nrw_path / 'all_items_053340002002.csv'
nrw_csv_v2 = nrw_path / 'all_items_053340002002_v2.csv'

aachen_path = pathlib.Path('../workflow/output/geotiff-aachen-z19/satellite')

airs_test_path = pathlib.Path('/storage/internal/data/r-pueblas/airs/test')
airs_image_path = airs_test_path / 'image'
airs_label_path = airs_test_path / 'label'

In [5]:
def intersect_and_union(pred_label, label, num_classes, ignore_index):
    mask = (label != ignore_index)
    pred_label = pred_label[mask]
    label = label[mask]

    intersect = pred_label[pred_label == label]
    area_intersect, _ = np.histogram(
        intersect, bins=np.arange(num_classes + 1))
    area_pred_label, _ = np.histogram(
        pred_label, bins=np.arange(num_classes + 1))
    area_label, _ = np.histogram(label, bins=np.arange(num_classes + 1))
    area_union = area_pred_label + area_label - area_intersect

    return area_intersect, area_union, area_pred_label, area_label

In [6]:
def test_model_rooftops(model, test_folder, num_classes, background_class, ignore_index, output=True):
    start = time.time()
    
    test_folder_image = test_folder / 'image'
    test_folder_label = test_folder / 'label'

    total_area_intersect = np.zeros((num_classes, ), dtype=float)
    total_area_union = np.zeros((num_classes, ), dtype=float)
    total_area_pred_label = np.zeros((num_classes, ), dtype=float)
    total_area_label = np.zeros((num_classes, ), dtype=float)

    for i, filename in enumerate(test_folder_image.glob('*.png')):
        if i>200: break

        image = cv2.imread(str(filename))
        label = cv2.imread(str(test_folder_label / filename.name))
        label = label[:,:,0]

        pred = prediction.segment_img(image, model, tile_size=512, stride=512, background_class=background_class)

        area_intersect, area_union, area_pred_label, area_label = \
                            intersect_and_union(pred, label, num_classes, ignore_index)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        if i<5 and output:
            f, axarr = plt.subplots(1,3, figsize=(10, 10))
            axarr[0].imshow(image)
            axarr[1].imshow(label)
            axarr[2].imshow(pred)
            if i==0:
                axarr[0].title.set_text('Image')
                axarr[1].title.set_text('Ground truth')
                axarr[2].title.set_text('aug_10e')

            plt.subplots_adjust(wspace=0.1, hspace=0.1)
            for ax in axarr:
                ax.axis('off')
            plt.show()

        iou = total_area_intersect / total_area_union
        if output: print(f'Processed image {filename.stem}, partial mean iou: {np.mean(iou[~np.isnan(iou)])}')

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    acc = total_area_intersect / total_area_label
    iou = total_area_intersect / total_area_union

    end = time.time()
    elapsed_time = (end - start)

    if output:
        print('\n')
        print(f'Elapsed time: {elapsed_time} seconds')
        print(f'Final IoU per class: {iou}')
        print(f'Final mean IoU: {np.mean(iou)}')
    
    return iou, np.mean(iou[~np.isnan(iou)]), elapsed_time

def test_model_rid(model, test_folder, num_classes, rid_background_class, background_class, ignore_index, output=True):
    start = time.time()
    
    test_folder_image = test_folder / 'image'
    test_folder_label = test_folder / 'label'

    total_area_intersect = np.zeros((num_classes, ), dtype=float)
    total_area_union = np.zeros((num_classes, ), dtype=float)
    total_area_pred_label = np.zeros((num_classes, ), dtype=float)
    total_area_label = np.zeros((num_classes, ), dtype=float)

    for i, filename in enumerate(test_folder_image.glob('*.png')):
        image = cv2.imread(str(filename))
        label = cv2.imread(str(test_folder_label / filename.name))
        label = label[:,:,0]
        
        label[label != rid_background_class] = 1
        label[label == rid_background_class] = 0

        pred = prediction.segment_img(image, model, tile_size=512, stride=512, background_class=background_class)

        area_intersect, area_union, area_pred_label, area_label = \
                            intersect_and_union(pred, label, num_classes, ignore_index)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        if i<5 and output:
            f, axarr = plt.subplots(1,3, figsize=(10, 10))
            axarr[0].imshow(image)
            axarr[1].imshow(label)
            axarr[2].imshow(pred)
            if i==0:
                axarr[0].title.set_text('Image')
                axarr[1].title.set_text('Ground truth')
                axarr[2].title.set_text('aug_10e')

            plt.subplots_adjust(wspace=0.1, hspace=0.1)
            for ax in axarr:
                ax.axis('off')
            plt.show()

        iou = total_area_intersect / total_area_union
        if output: print(f'Processed image {filename.stem}, partial mean iou: {np.mean(iou[~np.isnan(iou)])}')

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    acc = total_area_intersect / total_area_label
    iou = total_area_intersect / total_area_union

    end = time.time()
    elapsed_time = (end - start)

    if output:
        print('\n')
        print(f'Elapsed time: {elapsed_time} seconds')
        print(f'Final IoU per class: {iou}')
        print(f'Final mean IoU: {np.mean(iou)}')
    
    return iou, np.mean(iou[~np.isnan(iou)]), elapsed_time

def test_model_airs(model, test_folder, scale_percent, num_classes, background_class, ignore_index, output=True, n_samples=None):
    start = time.time()
    
    test_folder_image = test_folder / 'image'
    test_folder_label = test_folder / 'label'

    total_area_intersect = np.zeros((num_classes, ), dtype=float)
    total_area_union = np.zeros((num_classes, ), dtype=float)
    total_area_pred_label = np.zeros((num_classes, ), dtype=float)
    total_area_label = np.zeros((num_classes, ), dtype=float)

    for i, filename in enumerate(test_folder_image.glob('*.tif')):
        if n_samples and (n_samples<=i): break
        image = cv2.imread(str(filename))
        label = cv2.imread(str(test_folder_label / filename.name))
        label = label[:,:,0]
    
        width = int(image.shape[1] * scale_percent / 100)
        height = int(image.shape[0] * scale_percent / 100)
        dsize = (width, height)
        img = cv2.resize(image, dsize)
        label = cv2.resize(label, dsize)

        pred = prediction.segment_img(img, model, tile_size=512, stride=512, background_class=background_class)

        area_intersect, area_union, area_pred_label, area_label = \
                            intersect_and_union(pred, label, num_classes, ignore_index)
        total_area_intersect += area_intersect
        total_area_union += area_union
        total_area_pred_label += area_pred_label
        total_area_label += area_label

        if i<5 and output:
            f, axarr = plt.subplots(1,3, figsize=(10, 10))
            axarr[0].imshow(image)
            axarr[1].imshow(label)
            axarr[2].imshow(pred)
            if i==0:
                axarr[0].title.set_text('Image')
                axarr[1].title.set_text('Ground truth')
                axarr[2].title.set_text('aug_10e')

            plt.subplots_adjust(wspace=0.1, hspace=0.1)
            for ax in axarr:
                ax.axis('off')
            plt.show()

        iou = total_area_intersect / total_area_union
        if output: print(f'Processed image {filename.stem}, partial mean iou: {np.mean(iou[~np.isnan(iou)])}')

    all_acc = total_area_intersect.sum() / total_area_label.sum()
    acc = total_area_intersect / total_area_label
    iou = total_area_intersect / total_area_union

    end = time.time()
    elapsed_time = (end - start)

    if output:
        print('\n')
        print(f'Elapsed time: {elapsed_time} seconds')
        print(f'Final IoU per class: {iou}')
        print(f'Final mean IoU: {np.mean(iou)}')
    
    return iou, np.mean(iou[~np.isnan(iou)]), elapsed_time

# Testing INRIA

In [7]:
num_classes = 2
background_class = 0
ignore_index = -1

In [8]:
print(f'Testing model inria_aug_10e...')

iou, mean_iou, elapsed_time = test_model_rooftops(model_inria_aug_10e,
                                                         inria_val_folder,
                                                         num_classes,
                                                         background_class,
                                                         ignore_index,
                                                         output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_aug_10e...
Mean IoU: 0.847807326132997
IoU per class: [0.93641253 0.75920213]
Elapsed time: 138.40591669082642 seconds.


In [9]:
print(f'Testing model inria_raw_10e...')

iou, mean_iou, elapsed_time = test_model_rooftops(model_inria_raw_10e,
                                                         inria_val_folder,
                                                         num_classes,
                                                         background_class,
                                                         ignore_index,
                                                         output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_raw_10e...
Mean IoU: 0.8235779210729894
IoU per class: [0.92629019 0.72086566]
Elapsed time: 130.05559659004211 seconds.


In [10]:
print(f'Testing model inria_aug_5e...')

iou, mean_iou, elapsed_time = test_model_rooftops(model_inria_aug_5e,
                                                         inria_val_folder,
                                                         num_classes,
                                                         background_class,
                                                         ignore_index,
                                                         output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_aug_5e...
Mean IoU: 0.8465193867257228
IoU per class: [0.93559357 0.75744521]
Elapsed time: 170.12336587905884 seconds.


In [11]:
print(f'Testing model inria_raw_5e...')

iou, mean_iou, elapsed_time = test_model_rooftops(model_inria_raw_5e,
                                                         inria_val_folder,
                                                         num_classes,
                                                         background_class,
                                                         ignore_index,
                                                         output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_raw_5e...
Mean IoU: 0.8162042423986741
IoU per class: [0.92407889 0.70832959]
Elapsed time: 149.79505252838135 seconds.


# Testing RID

In [12]:
rid_background_class = 17

In [13]:
print(f'Testing model inria_aug_10e...')

iou, mean_iou, elapsed_time = test_model_rid(model_inria_aug_10e,
                                                  rid_test_folder,
                                                  num_classes,
                                                  rid_background_class,
                                                  background_class,
                                                  ignore_index,
                                                  output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_aug_10e...
Mean IoU: 0.7617473792970644
IoU per class: [0.87887034 0.64462442]
Elapsed time: 137.18511533737183 seconds.


In [14]:
print(f'Testing model inria_raw_10e...')

iou, mean_iou, elapsed_time = test_model_rid(model_inria_raw_10e,
                                                  rid_test_folder,
                                                  num_classes,
                                                  rid_background_class,
                                                  background_class,
                                                  ignore_index,
                                                  output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_raw_10e...
Mean IoU: 0.7661283942275854
IoU per class: [0.8793084  0.65294839]
Elapsed time: 135.04816675186157 seconds.


In [15]:
print(f'Testing model inria_aug_5e...')

iou, mean_iou, elapsed_time = test_model_rid(model_inria_aug_5e,
                                                  rid_test_folder,
                                                  num_classes,
                                                  rid_background_class,
                                                  background_class,
                                                  ignore_index,
                                                  output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_aug_5e...
Mean IoU: 0.7662252473894691
IoU per class: [0.88480345 0.64764705]
Elapsed time: 143.93829226493835 seconds.


In [16]:
print(f'Testing model inria_raw_5e...')

iou, mean_iou, elapsed_time = test_model_rid(model_inria_raw_5e,
                                                  rid_test_folder,
                                                  num_classes,
                                                  rid_background_class,
                                                  background_class,
                                                  ignore_index,
                                                  output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_raw_5e...
Mean IoU: 0.7763587105142358
IoU per class: [0.88934575 0.66337167]
Elapsed time: 139.5976917743683 seconds.


# AIRS

In [17]:
background_class = 0
num_classes = 2
scale_percent = 25

In [18]:
print(f'Testing model inria_aug_10e...')

iou, mean_iou, elapsed_time = test_model_airs(model_inria_aug_10e,
                                              airs_test_path,
                                              scale_percent,
                                              num_classes,
                                              background_class,
                                              ignore_index,
                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_aug_10e...
Mean IoU: 0.866490661102786
IoU per class: [0.9776904  0.75529092]
Elapsed time: 1617.1714451313019 seconds.


In [19]:
print(f'Testing model inria_raw_10e...')

iou, mean_iou, elapsed_time = test_model_airs(model_inria_raw_10e,
                                              airs_test_path,
                                              scale_percent,
                                              num_classes,
                                              background_class,
                                              ignore_index,
                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_raw_10e...
Mean IoU: 0.8465287582990798
IoU per class: [0.97347912 0.71957839]
Elapsed time: 1711.276927471161 seconds.


In [20]:
print(f'Testing model inria_aug_5e...')

iou, mean_iou, elapsed_time = test_model_airs(model_inria_aug_5e,
                                              airs_test_path,
                                              scale_percent,
                                              num_classes,
                                              background_class,
                                              ignore_index,
                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_aug_5e...
Mean IoU: 0.7752117536901475
IoU per class: [0.95550337 0.59492014]
Elapsed time: 1640.1877014636993 seconds.


In [21]:
print(f'Testing model inria_raw_5e...')

iou, mean_iou, elapsed_time = test_model_airs(model_inria_raw_5e,
                                              airs_test_path,
                                              scale_percent,
                                              num_classes,
                                              background_class,
                                              ignore_index,
                                              output=False)

print(f'Mean IoU: {mean_iou}')
print(f'IoU per class: {iou}')
print(f'Elapsed time: {elapsed_time} seconds.')

Testing model inria_raw_5e...
Mean IoU: 0.7754282304151523
IoU per class: [0.95534785 0.59550861]
Elapsed time: 1637.2719938755035 seconds.


# Check optimal scale

In [ ]:
print(f'Testing model inria_aug_10e...')

airs_gr = 0.075
grs = [2,1,0.75,0.5,0.4,0.3,0.2,0.1,0.075]
ious = []

for gr in grs:
    scale_percent = (airs_gr / gr) * 100
    print(f'Testing scale {scale_percent}%, gr {gr} m/pixel')
    iou, mean_iou, elapsed_time = test_model_airs(model_inria_aug_10e,
                                                  airs_test_path,
                                                  scale_percent,
                                                  num_classes,
                                                  background_class,
                                                  ignore_index,
                                                  output=False,
                                                  n_samples=10
                                                 )
    print(f'IoU: {iou}')
    ious.append(iou)

Testing model inria_aug_10e...
Testing scale 3.75%, gr 2 m/pixel
IoU: [0.92894031 0.09754744]
Testing scale 7.5%, gr 1 m/pixel
IoU: [0.92824647 0.22033324]
Testing scale 10.0%, gr 0.75 m/pixel
IoU: [0.94412707 0.39447996]
Testing scale 15.0%, gr 0.5 m/pixel
IoU: [0.96599575 0.62659586]
Testing scale 18.749999999999996%, gr 0.4 m/pixel
IoU: [0.97355512 0.70307351]
Testing scale 25.0%, gr 0.3 m/pixel
IoU: [0.97834271 0.75712884]
Testing scale 37.49999999999999%, gr 0.2 m/pixel
IoU: [0.97771722 0.75846735]
Testing scale 74.99999999999999%, gr 0.1 m/pixel


In [ ]:
with plt.rc_context({"xtick.major.pad": 0}):
    fig, ax = plt.subplots()

    ious_np = np.array(ious)

    width = 0.5

    scales = list(range(len(grs)))
    ax.bar(np.array(scales) - (width / 2), ious_np[:,0], width=width, edgecolor="white", linewidth=0)
    ax.bar(np.array(scales) - (width / 2), ious_np[:,1], width=width, edgecolor="white", linewidth=0)

    ax.set_xticks(scales)
    ax.set_xticklabels(grs, rotation=55)
    
    ax.set_xlabel('Ground resolution (m/pixel)')
    ax.set_ylabel('Intersection over Union')
    ax.legend(['Background', 'Rooftops'])

    fig.tight_layout()
    plt.show()